In [27]:
#Importing Python packages
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

In [28]:
#Importing ACSdata

ACSdata = pd.read_csv('data/ACSdata.csv', sep=',', header=0, skipinitialspace=True)

In [29]:
ACSdata.shape

(3190040, 127)

In [30]:
#EXPLORING THE DATA

#Generating summary statistics - age
ACSdata['AGEP'].describe().astype('int64')

#Age range (0-96), data set needs to be filtered to include persons of marriageable age(> 18)

count    3190040
mean          41
std           23
min            0
25%           21
50%           42
75%           60
max           96
Name: AGEP, dtype: int64

In [31]:
#Rows in which value of AGEP column in  less than 18
is_lt18 = ACSdata.apply(lambda x: True if x['AGEP'] < 18 else False, axis=1)

#Count number of True in the series
numofRows = len(is_lt18[is_lt18 ==True].index)

print('Number of Rows in ACSdata in which Age < 18:', numofRows)

Number of Rows in ACSdata in which Age < 18: 659314


In [32]:
#Wrangling - Removing persons from the dataset who are < 18 years and creating a new df (ACSmarry)
ACSmarry = ACSdata[ACSdata.AGEP >= 18]
print(ACSmarry.shape)

(2530726, 127)


In [33]:
#EXPLORING THE DATAFRAME (ACSmarry)
#Generating frequency table - marriage
ACSmarry['MAR'].value_counts()

1    1360300
5     671044
3     285664
2     169374
4      44344
Name: MAR, dtype: int64

In [34]:
#Printing ACSmarry to csv

#fields = ['AGEP','MAR','SERIALNO','SPORDER']
#Create a sample of top 100 to explore a smaller file size
#ACSsample = ACSmarry.head(10000)
# update to the path on your local drive
#ACSsample.to_csv(r'testdata/sample1.csv', columns=fields)

In [35]:
#Using ADJINC to calculate wages/earning and income variables
#Note--> this code will need to be adjusted for the multi-year file

ACSmarry['WAGES'] = ACSmarry.WAGP * (ACSmarry.ADJINC / 1000000 )
ACSmarry['INCOME'] = ACSmarry.PINCP * (ACSmarry.ADJINC / 1000000 )
ACSmarry['EARN'] = ACSmarry.PERNP * (ACSmarry.ADJINC / 1000000 )

C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\steph\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.

In [36]:
ACSmoney = ACSmarry[['WAGES', 'WAGP', 'INCOME', 'PINCP', 'EARN', 'PERNP']]

ACSmoney.describe().astype('int64')

,WAGES,WAGP,INCOME,PINCP,EARN,PERNP
count,2530726,2530726,2530726,2530726,2530726,2530726
mean,31072,30728,43202,42724,33157,32791
std,55479,54865,62819,62123,58403,57757
min,0,0,-9201,-9100,-9100,-9000
25%,0,0,9950,9840,0,0
50%,10213,10100,26290,26000,14156,14000
75%,43481,43000,53593,53000,45503,45000
max,744235,736000,1580488,1563000,1274098,1260000


In [37]:
#print(ACSmarry[['ADJINC', 'WAGP', 'WAGES', 'PINCP', 'INCOME', 'PERNP', 'EARN']].head())

In [38]:
#Running summary statistics for continuous variables
ACScont = ACSmarry[['WAGES', 'INCOME', 'EARN', 'AGEP', 'JWMNP', 'WKHP']]

ACScont.describe().astype('int64')

,WAGES,INCOME,EARN,AGEP,JWMNP,WKHP
count,2530726,2530726,2530726,2530726,1362382,1627454
mean,31072,43202,33157,49,27,38
std,55479,62819,58403,18,23,12
min,0,-9201,-9100,18,1,1
25%,0,9950,0,34,12,35
50%,10213,26290,14156,50,20,40
75%,43481,53593,45503,64,35,44
max,744235,1580488,1274098,96,160,99


In [39]:
#Frequency tables and possible transformation of categorical variables

#Citizenship status

ACSmarry['CIT'].value_counts(sort=True) 

1    2135403
4     197130
5     160061
3      23517
2      14615
Name: CIT, dtype: int64

In [40]:
#Transformaing--> Citizen (yes/no)
ACSmarry.loc[ACSmarry.CIT == 5, 'CITIZEN'] = 0
ACSmarry.loc[ACSmarry.CIT != 5, 'CITIZEN'] = 1

ACSmarry['CITIZEN'].value_counts(sort=True) 

1.0    2370665
0.0     160061
Name: CITIZEN, dtype: int64

In [41]:
ACSmarry['MAR'].value_counts(sort=True) #Marital status

1    1360300
5     671044
3     285664
2     169374
4      44344
Name: MAR, dtype: int64

In [42]:
ACSmarry['MAR'].value_counts(sort=True, normalize=True) * 100 #Marital status

1    53.751374
5    26.515869
3    11.287828
2     6.692704
4     1.752224
Name: MAR, dtype: float64

In [43]:
#Transforming number of times married
ACSmarry['MARHT'].value_counts(sort=True) #Number of times married

1.0    1385866
2.0     366893
3.0     106923
Name: MARHT, dtype: int64

In [44]:
ACSmarry['MARHT'].value_counts(sort=True, normalize=True) * 100 

1.0    74.521666
2.0    19.728803
3.0     5.749531
Name: MARHT, dtype: float64

In [45]:
#Marital status by the number of times married
pd.crosstab(ACSmarry.MAR, ACSmarry.MARHT, margins=True)

MARHT,1.0,2.0,3.0,All
MAR,,,,
1,1031074,257976,71250,1360300
2,125754,33561,10059,169374
3,196988,66202,22474,285664
4,32050,9154,3140,44344
All,1385866,366893,106923,1859682


In [46]:
#Creating variable: Previously married (# of times)
ACSmarry.loc[ACSmarry.MAR == 5, 'MAR_TIMES'] = 0 #Married zero times
ACSmarry.loc[(ACSmarry.MAR != 5) & (ACSmarry.MARHT == 1), 'MAR_TIMES'] = 1 #married 1 time
ACSmarry.loc[(ACSmarry.MAR != 5) & (ACSmarry.MARHT != 1), 'MAR_TIMES'] = 2 #married more than 1 times

ACSmarry['MAR_TIMES'].value_counts(sort=True) 

1.0    1385866
0.0     671044
2.0     473816
Name: MAR_TIMES, dtype: int64

In [47]:
#Mobility:  (lived here 1 year aga)
ACSmarry['MIG'].value_counts(sort=True)

1.0    2186818
3.0     329380
2.0      14528
Name: MIG, dtype: int64

In [48]:
#Transforming MIG to "Mover, or changed locations 1 year or less ago"
ACSmarry.loc[ACSmarry.MIG != 1, 'MOVER'] = 1 #Moved, changed location
ACSmarry.loc[ACSmarry.MIG == 1, 'MOVER'] = 0 #Did not move or change location

ACSmarry['MOVER'].value_counts(sort=True)

0.0    2186818
1.0     343908
Name: MOVER, dtype: int64

In [49]:
ACSmarry['MOVER'].value_counts(sort=True, normalize=True) * 100

0.0    86.410698
1.0    13.589302
Name: MOVER, dtype: float64

In [50]:
#Transforming educational attainment variable (SCHL)

ACSmarry['SCHL'].value_counts(sort=True)

16.0    593362
21.0    474195
19.0    385417
22.0    210462
20.0    208034
18.0    183853
17.0    100775
14.0     57847
23.0     51635
15.0     48886
13.0     41492
1.0      35862
24.0     34357
12.0     31612
11.0     26780
9.0      19603
10.0      8416
8.0       5445
6.0       4795
7.0       3696
5.0       2119
4.0        973
3.0        583
2.0        527
Name: SCHL, dtype: int64

In [51]:
ACSmarry.loc[ACSmarry.SCHL < 16, 'EDUCATION'] = 0 #No high school diploma or GED
ACSmarry.loc[(ACSmarry.SCHL == 16) | (ACSmarry.SCHL == 17), 'EDUCATION'] = 1 #HS diploma or GED
ACSmarry.loc[(ACSmarry.SCHL == 18) | (ACSmarry.SCHL == 19) | (ACSmarry.SCHL == 20), 'EDUCATION'] = 2 #less thhan a college degree or an associate's
ACSmarry.loc[(ACSmarry.SCHL == 21), 'EDUCATION'] = 3 #Bachelor's degree
ACSmarry.loc[(ACSmarry.SCHL == 22) | (ACSmarry.SCHL == 23) | (ACSmarry.SCHL == 24), 'EDUCATION'] = 4 #Graduate degree

ACSmarry['EDUCATION'].value_counts(sort=True)

2.0    777304
1.0    694137
3.0    474195
4.0    296454
0.0    288636
Name: EDUCATION, dtype: int64

In [52]:
#Marital status by educational attainment
pd.crosstab(ACSmarry.MAR, ACSmarry.EDUCATION, margins=True)

EDUCATION,0.0,1.0,2.0,3.0,4.0,All
MAR,,,,,,
1,126647,339053,390516,296056,208028,1360300
2,35041,63118,41650,18079,11486,169374
3,31425,85992,96490,43508,28249,285664
4,9188,14089,13568,4827,2672,44344
5,86335,191885,235080,111725,46019,671044
All,288636,694137,777304,474195,296454,2530726


In [53]:
#Race by Marital status

pd.crosstab(ACSmarry.MAR, ACSmarry.RAC1P, margins=True)

RAC1P,1,2,3,4,5,6,7,8,9,All
MAR,,,,,,,,,,
1,1116910,80954,8567,558,1335,85886,2009,41848,22233,1360300
2,138422,17937,1417,102,231,6325,213,2706,2021,169374
3,228406,32913,2873,136,444,7265,349,7162,6116,285664
4,28497,9335,567,42,100,1500,86,2962,1255,44344
5,457907,107909,7831,695,1219,37490,1373,33142,23478,671044
All,1970142,249048,21255,1533,3329,138466,4030,87820,55103,2530726


In [54]:
#Condensing the race categories

#ACSmarry.loc[ACSmarry.RACP1 == 1, 'RACE'] = 1 #White
#ACSmarry.loc[(ACSmarry.RACP1 == 2) 'RACE'] = 2 #Black, African American
#ACSmarry.loc[(ACSmarry.RACP1 == 3) | (ACSmarry.RACP1 == 4) | (ACSmarry.RACP1 ==5), 'RACE'] = 3 #American Indian/Alaskan Native
#ACSmarry.loc[(ACSmarry.RACP1 == 6) 'RACE'] = 4 #Asian
#ACSmarry.loc[(ACSmarry.RACP1 == 7) 'RACE'] = 5 #Native Hawaiian/Pacific Islander
#ACSmarry.loc[(ACSmarry.RACP1 == 8) 'RACE'] = 2 #Black, African American
#ACSmarry.loc[(ACSmarry.SCHL == 18) | (ACSmarry.SCHL == 19) | (ACSmarry.SCHL == 20), 'EDUCATION'] = 2 #less thhan a college degree or an associate's
#ACSmarry.loc[(ACSmarry.SCHL == 21), 'EDUCATION'] = 3 #Bachelor's degree
#ACSmarry.loc[(ACSmarry.SCHL == 22) | (ACSmarry.SCHL == 23) | (ACSmarry.SCHL == 24), 'EDUCATION'] = 4 #Graduate degree

#ACSmarry['RACE'] = ACSmarry['RACP1']

#Setting the condition to create American Indian/Alaskan
#conds = 

#Setting the value for the aforemnetioned condition
#codes = [3]


# Use np.select with a default of 3 (your "else" value)    
#df['progress'] = np.select(conds, choices, default = 3)

In [55]:
#contingency_table = pd.crosstab(
 #   workhour_by_sex['sex'],
  #  workhour_by_sex['hours_per_week_categories'],
   # margins = True
#)
#contingency_table

In [56]:
#Assigns the frequency values
#malecount = contingency_table.iloc[0][0:6].values
#femalecount = contingency_table.iloc[1][0:6].values

#Plots the bar chart
#fig = plt.figure(figsize=(10, 5))
#sns.set(font_scale=1.8)
#categories = ["0-9","10-19","20-29","30-39","40-49","50+"]
#p1 = plt.bar(categories, malecount, 0.55, color='#d62728')
#p2 = plt.bar(categories, femalecount, 0.55, bottom=malecount)
#plt.legend((p2[0], p1[0]), ('Male', 'Female'))
#plt.xlabel('Hours per Week Worked')
#plt.ylabel('Count')
#plt.show()

In [57]:
#Generate list of weights of the format PWGTP1 to PWGTP80
#weight_index = range(1,81,1)
#weight_list = ['PWGTP'+str(i) for i in weight_index]
#print(weight_list)

#THIS IS NOT 100% CORRECT.  THE WEIGHTS ALREADY EXIST ON THE CSV FILES
